# Supervised Fine-Tuning with SFTTrainer

In [1]:
# Authenticate to Hugging Face
from huggingface_hub import login
from dotenv import load_dotenv
import os

# 加载.env文件
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
# 导入必要的包
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
cache_dir = "../../huggingface"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    cache_dir=cache_dir
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, cache_dir=cache_dir)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "xcs_module"]

# Generate with the base model
尝试没有聊天模板的基本模型

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


# 数据集准备
加载一个示例数据集并对其进行格式化以进行训练。数据集应使用输入-输出对进行结构化，其中每个输入都是一个提示，输出是模型的预期响应。

TRL将根据模型的聊天模板设置输入消息的格式。它们需要表示为具有键的字典列表：role 和 content。

In [4]:
# Load a sample dataset
from datasets import load_dataset

ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

ds

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

In [5]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. 
# Refer to the [module](../chat_templates.md)
#EXAMPLE
def formatting_func(example):
    text = (f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
            f"{example['question']}<|eot_id|><|start_header_id|>"
            f"assistant<|end_header_id|>\n\n{example['answer']}<|eot_id|>")
    return {"text" : text}

# Configuring the SFTTrainer
SFTTrainer 配置了控制训练过程的各种参数。这些指标包括训练步骤的数量、批量大小、学习率和评估策略。根据特定要求和计算资源调整这些参数

In [6]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # 根据数据集大小和所需的训练持续时间进行调整
    per_device_train_batch_size=4,  # 根据GPU内存容量设置
    learning_rate=5e-5,  # 微调的一般性起点
    logging_steps=10,  # 日志训练指标的频率
    save_steps=100,  # 保存模型检查点的频率
    evaluation_strategy="steps",  # 定期评估模型
    eval_steps=50,  # 评估频率
    use_mps_device=(
        True if device == "mps" else False
    ),  # 使用 MPS 进行混合精度训练
    hub_model_id=finetune_name,  # 为模型设置一个唯一的名称
)

class CustomTrainer(SFTTrainer):
    def log(self, logs):
        super().log(logs)
        if "loss" in logs:
            print(f"Train Loss: {logs['loss']:.4f}")
        if "eval_loss" in logs:
            print(f"Eval Loss: {logs['eval_loss']:.4f}")


# Initialize the SFTTrainer
trainer = CustomTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

/Users/landon/Anaconda/anaconda3/envs/llm/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/Users/landon/Anaconda/anaconda3/envs/llm/lib/python3.8/site-packages/transformers/training_args.py:2248: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
/Users/landon/Anaconda/anaconda3/envs/llm/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

/Users/landon/Anaconda/anaconda3/envs/llm/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
max_steps is given, it will override any value given in num_train_epochs


## Training the Model 
配置好trainer后，现在可以继续训练模型。训练过程将涉及迭代数据集、计算损失以及更新模型的参数以最大限度地减少这种损失。

In [7]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
huggingface/tokenizers: 

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.6774, 'grad_norm': 2.7204604148864746, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
Train Loss: 1.6774
{'loss': 1.2563, 'grad_norm': 2.644317150115967, 'learning_rate': 4.9e-05, 'epoch': 0.04}
Train Loss: 1.2563
{'loss': 1.2339, 'grad_norm': 2.39457106590271, 'learning_rate': 4.85e-05, 'epoch': 0.05}
Train Loss: 1.2339
{'loss': 1.161, 'grad_norm': 2.5026965141296387, 'learning_rate': 4.8e-05, 'epoch': 0.07}
Train Loss: 1.1610
{'loss': 1.0657, 'grad_norm': 2.2998547554016113, 'learning_rate': 4.75e-05, 'epoch': 0.09}
Train Loss: 1.0657


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.158974528312683, 'eval_runtime': 8.9921, 'eval_samples_per_second': 13.234, 'eval_steps_per_second': 1.668, 'epoch': 0.09}
Eval Loss: 1.1590
{'loss': 1.1566, 'grad_norm': 2.288527727127075, 'learning_rate': 4.7e-05, 'epoch': 0.11}
Train Loss: 1.1566
{'loss': 1.1648, 'grad_norm': 2.0123109817504883, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
Train Loss: 1.1648
{'loss': 1.1244, 'grad_norm': 2.1283881664276123, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
Train Loss: 1.1244
{'loss': 1.086, 'grad_norm': 2.224285125732422, 'learning_rate': 4.55e-05, 'epoch': 0.16}
Train Loss: 1.0860
{'loss': 1.1116, 'grad_norm': 2.0807411670684814, 'learning_rate': 4.5e-05, 'epoch': 0.18}
Train Loss: 1.1116


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.1240628957748413, 'eval_runtime': 5.2858, 'eval_samples_per_second': 22.513, 'eval_steps_per_second': 2.838, 'epoch': 0.18}
Eval Loss: 1.1241
{'loss': 1.0621, 'grad_norm': 2.0502209663391113, 'learning_rate': 4.4500000000000004e-05, 'epoch': 0.19}
Train Loss: 1.0621
{'loss': 1.1234, 'grad_norm': 2.0778651237487793, 'learning_rate': 4.4000000000000006e-05, 'epoch': 0.21}
Train Loss: 1.1234
{'loss': 1.0921, 'grad_norm': 1.9966037273406982, 'learning_rate': 4.35e-05, 'epoch': 0.23}
Train Loss: 1.0921
{'loss': 1.0627, 'grad_norm': 2.129854679107666, 'learning_rate': 4.3e-05, 'epoch': 0.25}
Train Loss: 1.0627
{'loss': 1.0624, 'grad_norm': 2.0639731884002686, 'learning_rate': 4.25e-05, 'epoch': 0.27}
Train Loss: 1.0624


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0954866409301758, 'eval_runtime': 4.9285, 'eval_samples_per_second': 24.145, 'eval_steps_per_second': 3.044, 'epoch': 0.27}
Eval Loss: 1.0955
{'loss': 1.0679, 'grad_norm': 2.023447036743164, 'learning_rate': 4.2e-05, 'epoch': 0.28}
Train Loss: 1.0679


RuntimeError: MPS backend out of memory (MPS allocated: 11.50 GB, other allocations: 8.74 GB, max allowed: 20.40 GB). Tried to allocate 198.75 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

## Bonus Exercise: Generate with fine-tuned model

🐕 使用微调模型生成响应

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
prompt = "Write a haiku about programming"

inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))